In [3]:

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
import os
import pickle

# Paths and constants
dataset_path = '../dataset/'
img_height, img_width = 224, 224
batch_size = 32
epochs = 30  # Increased epochs for better performance
model_path = '../models/mobilenetv2_model.h5'

# Data generators (already written)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2],  # Added brightness range for more robustness
    validation_split=0.2
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    directory=dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

validation_generator = val_datagen.flow_from_directory(
    directory=dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    subset='validation'
)

# Get number of classes
num_classes = len(train_generator.class_indices)
print("Class Indices:", train_generator.class_indices)

# Build MobileNetV2 model
base_model = MobileNetV2(input_shape=(img_height, img_width, 3),
                         include_top=False,
                         weights='imagenet')

base_model.trainable = False  # Freeze the base model initially

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)  # Adding batch normalization
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = LearningRateScheduler(lambda epoch: 0.0001 * 0.9 ** epoch)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, lr_scheduler, reduce_lr]
)

# Fine-tuning (optional): Unfreeze more layers of MobileNetV2
base_model.trainable = True
fine_tune_at = 100  # Unfreeze from this layer onward, can adjust further
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile the model after unfreezing layers
model.compile(optimizer=Adam(learning_rate=0.00001),  # Lower learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training the model with fine-tuning
history_finetune = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, lr_scheduler, reduce_lr]
)

# Save the model
os.makedirs('../models', exist_ok=True)
model.save(model_path)
print(f"✅ Model saved to {model_path}")

# Save history
with open('../models/history.pkl', 'wb') as f:
    pickle.dump(history_finetune.history, f)

print("✅ History saved successfully! 🎉")


Found 2364 images belonging to 18 classes.
Found 588 images belonging to 18 classes.
Class Indices: {'01_healthy_paddy': 0, '02_leaf_blast_paddy': 1, '03_bacterial_leaf_blight_paddy': 2, '04_healthy_banana': 3, '05_cordana_banana': 4, '06_sigatoka_banana': 5, '07_Healthy_sugarcane': 6, '08_Mosaic_sugarcane': 7, '09_RedRot_sugarcane': 8, '10_healthy_leaf_groundnut': 9, '11_early_leaf_spot_groundnut': 10, '12_Rust_groundnut': 11, '13_Healthy blackgram': 12, '14_Yellow Mosaic blackgram': 13, '15_Powdery_Mildew_blackgram': 14, '16_tomato_healthy': 15, '17_Tomato_Yellow_Leaf_Curl_Virus': 16, '18_Early_blight_tomato': 17}
Epoch 1/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 56s 749ms/step - accuracy: 0.2381 - loss: 2.6863 - val_accuracy: 0.5625 - val_loss: 1.5871 - learning_rate: 1.0000e-04
Epoch 2/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 10s 131ms/step - accuracy: 0.5000 - loss: 1.5387 - val_accuracy: 0.5677 - val_loss: 1.5746 - learning_rate: 9.0000e-05
Epoch 3/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 

✅ Model saved to ../models/mobilenetv2_model.h5
✅ History saved successfully! 🎉


In [ ]:
!pip install matplotlib

In [6]:
!pip install Pillow


Found 2301 images belonging to 6 classes.
Found 573 images belonging to 6 classes.
Class Indices: {'Banana': 0, 'Blackgram': 1, 'Groundnut': 2, 'Paddy': 3, 'Sugarcane': 4, 'Tomato': 5}


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [10]:
!pip uninstall -y pillow

Found existing installation: pillow 11.2.1
Uninstalling pillow-11.2.1:
  Successfully uninstalled pillow-11.2.1


You can safely remove it manually.


In [11]:
!pip install pillow

  Using cached pillow-11.2.1-cp39-cp39-win_amd64.whl.metadata (9.1 kB)
Using cached pillow-11.2.1-cp39-cp39-win_amd64.whl (2.7 MB)


Found 2364 images belonging to 18 classes.
Found 588 images belonging to 18 classes.
Class Indices: {'01_healthy_paddy': 0, '02_leaf_blast_paddy': 1, '03_bacterial_leaf_blight_paddy': 2, '04_healthy_banana': 3, '05_cordana_banana': 4, '06_sigatoka_banana': 5, '07_Healthy_sugarcane': 6, '08_Mosaic_sugarcane': 7, '09_RedRot_sugarcane': 8, '10_healthy_leaf_groundnut': 9, '11_early_leaf_spot_groundnut': 10, '12_Rust_groundnut': 11, '13_Healthy blackgram': 12, '14_Yellow Mosaic blackgram': 13, '15_Powdery_Mildew_blackgram': 14, '16_tomato_healthy': 15, '17_Tomato_Yellow_Leaf_Curl_Virus': 16, '18_Early_blight_tomato': 17}
Epoch 1/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 45s 568ms/step - accuracy: 0.1728 - loss: 2.7465 - val_accuracy: 0.5486 - val_loss: 1.7349 - learning_rate: 1.0000e-04
Epoch 2/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - accuracy: 0.7188 - loss: 1.4922 - val_accuracy: 0.5486 - val_loss: 1.7231 - learning_rate: 9.0000e-05
Epoch 3/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 40s 544ms/step - accuracy:

✅ Model saved to ../models/mobilenetv2_model.h5
✅ History saved successfully! 🎉


In [14]:
pip uninstall pillow PIL PILLOW -y


Found existing installation: pillow 11.2.1
Uninstalling pillow-11.2.1:
  Successfully uninstalled pillow-11.2.1
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install Pillow



  Using cached pillow-11.2.1-cp39-cp39-win_amd64.whl.metadata (9.1 kB)
Using cached pillow-11.2.1-cp39-cp39-win_amd64.whl (2.7 MB)


In [16]:
from PIL import Image
print("✅ Pillow is working!")


✅ Pillow is working!


In [2]:
pip install scipy


   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   - -------------------------------------- 2.1/46.2 MB 11.8 MB/s eta 0:00:04
   ---- ----------------------------------- 5.2/46.2 MB 13.3 MB/s eta 0:00:04
   ------- -------------------------------- 8.7/46.2 MB 14.5 MB/s eta 0:00:03
   ---------- ----------------------------- 11.8/46.2 MB 14.8 MB/s eta 0:00:03
   -------------- ------------------------- 16.3/46.2 MB 16.3 MB/s eta 0:00:02
   ---------------- ----------------------- 19.1/46.2 MB 15.9 MB/s eta 0:00:02
   ------------------- -------------------- 22.5/46.2 MB 16.0 MB/s eta 0:00:02
   ---------------------- ----------------- 26.2/46.2 MB 16.1 MB/s eta 0:00:02
   ------------------------- -------------- 29.4/46.2 MB 16.1 MB/s eta 0:00:02
   --------------------------- ------------ 32.0/46.2 MB 15.9 MB/s eta 0:00:01
   ------------------------------ --------- 35.1/46.2 MB 15.6 MB/s eta 0:00:01
   --------------------------------- ------ 38.3/46.2 MB 15.6 MB